In [145]:
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import StratifiedKFold
import prune
from sklearn.metrics import roc_auc_score
from imblearn.ensemble import RUSBoostClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import EasyEnsembleClassifier

In [150]:
name = 'data/ecoli1.dat'
data = np.genfromtxt(fname = name, comments='@', delimiter=',', autostrip=True)
X = data[:,:-1]
data = pd.read_csv(name, comment='@', header = None, delimiter=',', delim_whitespace=True)
y = data.iloc[:,-1].values

In [151]:
enc = LabelEncoder()
y = enc.fit_transform(y)

In [154]:
bagging = EasyEnsembleClassifier(n_estimators= 30)

In [155]:
metrics = np.zeros((2,2))
skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    bagging.fit(X_train, y_train)
    metrics[0,:] += np.array([roc_auc_score(y_test, bagging.predict(X_test)), geometric_mean_score(y_test, bagging.predict(X_test))]) 
    bagging.estimators_ = prune.reduce_error_GM(bagging, X_train, y_train)
    metrics[1,:] += np.array([roc_auc_score(y_test, bagging.predict(X_test)), geometric_mean_score(y_test, bagging.predict(X_test))]) 
metrics /= skf.n_splits
print(metrics)

[[0.84841252 0.83717951]
 [0.86559201 0.85557676]]
